In [8]:
from TimeTagger import FileReader
import numpy as np
import easygui
import pandas as pd
import matplotlib.pyplot as plt



We know that H and V got swapped in BOB so I must correct for it in code. 

In [9]:
#GLOBAL DEFINITIONS
aH = 5
aV = 6
bH = 1
bV = 2
cH = 3
cV = 4

H=0
V=1

In [10]:
#folder = r'C:\Data\\'
#folder = "/media/tanvir/SpooqyLab/Table_Top_Demo/1stOct2020"
#folder = "./test"
#folder = "/home/tanvir/work/Programming/gProj/Satellite_QKD/data_source"
#folder = "/home/tanvir/work/Programming/gProj/qkd-table-top-v2/qutools"
#folder = "/home/tanvir/work/Programming/gProj/qkd-table-top-v2/test"
#folder = "/home/tanvir/work/Programming/gProj/qkd-table-top-v2/swabian/newmatch11Jun21"
folder = "./VerifiableRNG_Data/logSwabian20220221-1433/"
#file = '2.97mW_1.1.ttbin'
#file = "onlyB.1.ttbin"
#file = "dark1.ttbin"
#file = "3_88mW00dB.1.ttbin"
file ="logSwabian20220221-1433.ttbin"
filepath = folder + '/' + file

#filepath = easygui.fileopenbox(default = folder)






In [11]:
def correct_timestamp(stamps,detectors,step_size_ps=250,sH=0,sV=0):
    cstamp=[]
    ts = 0
    for (s,d) in zip (stamps,detectors):
        ts = s
        if(d==H): 
            ts+=sH*step_size_ps
        elif(d==V):
            ts+=sV*step_size_ps 
        cstamp.append(ts)
        
    #print (step_size_ps*sH,step_size_ps*sV)
    
    return cstamp

In [12]:
def match_coincidence(at,ad,bt,bd,cwindow,shift):
    ia = 0
    ib = 0
    acoin_indexes = []
    bcoin_indexes = []
    while (ia<len(at) and ib <len(bt)):
        
        diff = bt[ib]- at[ia] + shift

        if (abs(diff)<=cwindow):
            #coin_count +=1
            #if ad[ia]==bd[ib]:
            acoin_indexes.append(ia)
            bcoin_indexes.append(ib)
            ia+=1
            ib+=1
            #continue
        elif (diff>cwindow):
            ia+=1
            #continue
        elif( diff <-cwindow):
            ib+=1
            #continue
        else:
            print ("ERROR! I should not be here")
            break
    return acoin_indexes,bcoin_indexes



In [13]:
def generate_tripartite_output(astamps,achannel, bstamps,bchannel,cstamps,cchannel, 
                               ab_indexes,ba_indexes, ac_indexes, ca_indexes):
    columnA=[]
    columnB=[]
    columnC=[]
    iab = 0
    
    
    iac = 0
    
    #match_iab=None
    #match_iba=None
    
    #match_iac=None
    #match_ica=None
    a=0
    b=0
    c=0
    while(iab < len(ab_indexes) and iac <len(ac_indexes)):
        match_iab = ab_indexes[iab]
        match_iba = ba_indexes[iab]
        
        match_iac = ac_indexes[iac]
        match_ica = ca_indexes[iac]
        

        if match_iab < match_iac: 
            #process tag_ib
            #a = 0 # phi+ entanglement brunch
            a=0
            
            b = achannel[match_iab]
            c = bchannel[match_iba]
            
            iab+=1
        elif match_iab > match_iac:
            #process gat_ic
            a = 1
            b = achannel[match_iac]
            c = 1- cchannel[match_ica]
            iac +=1
        else:
            iab+=1
            iac+=1
        columnA.append(a)
        columnB.append(b)
        columnC.append(c)
    
    return columnA,columnB,columnC
    

In [17]:
def processSwabianTimeStamps(fileIn, num_sets=1):
    
        file_reader = FileReader(fileIn)
               
        loop_i = 0
        while file_reader.hasData():
            #for loop_i in range(2):
            loop_i +=1
            print("Processing set: ",loop_i,end=' ')
            stamps = []
            channel = []
 
            #n_events = 100000; # Number of events to read at once
            n_events = 1000000; # Number of events to read at once
            
            # Read at most n_events.
            data = file_reader.getData(n_events);  # data is an instance of TimeTagStreamBuffer

            channelTmp = data.getChannels()
            stampsTmp = data.getTimestamps()
          #  overflow_types = data.getEventTypes()   # TimeTag = 0, Error = 1, OverflowBegin = 2, OverflowEnd = 3, MissedEvents = 4
          #  missed_events = data.getMissedEvents()
            
            for i in range(len(channelTmp)):
                channel.append(channelTmp[i])
                stamps.append(stampsTmp[i])
            #break
            #if stamps[-1] - stamps[0] > t:
            #    break
        
            #seperate out alice's, bob's and charlie's events
            astamps = []
            achannel = []
            bstamps = []
            bchannel = []
            cstamps = []
            cchannel = []

            for (s,c) in zip(stamps, channel):
                if (c==aH or c==aV):
                    astamps.append(s+2000)
                    achannel.append(c-aH)
                elif (c==bH or c==bV):
                    bstamps.append(s)
                    bchannel.append(c-bH)
                elif (c==cH or c==cV):
                    cstamps.append(s)
                    cchannel.append(c-cH)
                else :
                    print ("Error! Unknown channel: ",c)

            bstamps_corrected = correct_timestamp(bstamps,bchannel,step_size_ps=250,sH=0,sV=2)
            cstamps_corrected = correct_timestamp(cstamps,cchannel,step_size_ps=250,sH=-1,sV=2)
            
            #ab_indexes,bc_indexes = match_coincidence(astamps,achannel,bstamps_corrected,bchannel,750,4*250)
            ab_indexes,ba_indexes = match_coincidence(astamps,achannel,bstamps_corrected,bchannel,750,4*250)
            
            #ac_indexes,cc_indexes = match_coincidence(astamps,achannel,cstamps_corrected,cchannel,750,4*250)
            ac_indexes,ca_indexes = match_coincidence(astamps,achannel,cstamps_corrected,cchannel,750,4*250)
            
            A,B,C=generate_tripartite_output(astamps,achannel, bstamps,bchannel,cstamps,cchannel, 
                               ab_indexes,ba_indexes, ac_indexes, ca_indexes)
            
            df = pd.DataFrame()
            df['columnA'] = A
            df['columnB'] = B
            df['columnC'] = C
            df.to_csv("./output/PQRNG_"+str(loop_i)+".csv",index=False)
            print ("Done.")
            if loop_i == num_sets:
                break

In [ ]:
processSwabianTimeStamps(filepath,4)

Processing set:  1 Done.
Processing set:  2 Done.
Processing set:  3 